In [1]:
"""
@author: albertigno

"""

from MyDataset import *
import torch, time, os
import torch.nn as nn
import torch.nn.functional as F
#import networkx as nx
import matplotlib.pyplot as plt
#from matplotlib.gridspec import GridSpec

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ('Running on: {}'.format(device))

Running on: cuda:0


In [2]:
thresh = 0.3
batch_size = 256 # default 256
learning_rate = 1e-4 # default 1e-4
time_window = 50 # shd 50, nmnist 25-30
dataset_path = r'./../../datasets'

In [3]:
train_path = dataset_path+'/shd_digits/shd_train.h5'
test_path = dataset_path+'/shd_digits/shd_test.h5'
# load datasets
print("loading test set...")
test_dataset = MyDataset(test_path, 'hd_digits', time_window, device)
print("loading training set...")
train_dataset = MyDataset(train_path, 'hd_digits', time_window, device)
print("loading data with pytorch")
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False, drop_last=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)

loading test set...
num sample: 2264
torch.Size([2264, 50, 700]) torch.Size([2264, 20])
loading training set...
num sample: 8156
torch.Size([8156, 50, 700]) torch.Size([8156, 20])
loading data with pytorch


In [4]:
from snn_models import *
%load_ext autoreload
%autoreload 1
%aimport snn_models

In [24]:
# parameter grid
num_epochs = 200
import numpy as np

tau_m = [0.8305, 'adp']
num_hidden = [128, 256, 512, 1024, 2048]
delay_mode = ['nodelay','delay']
vreset = [0.0, 0.1]

num = len(tau_m)*len(num_hidden)*len(delay_mode)*len(vreset)

x, y, z, w = np.meshgrid(tau_m, num_hidden, delay_mode, vreset)
x=x.reshape(num)
y=y.reshape(num)
z=z.reshape(num)
w=w.reshape(num)

In [25]:
for i in range(num):

    if x[i]!='adp':
        tau_m = float(x[i]) 
    else:
        tau_m = x[i]
    num_hidden = int(y[i])
    delay_mode = z[i]
    vreset = float(w[i])

    #tau_m = 0.8305
    #snn = RSNN_delay(d='shd', num_hidden=128, thresh=0.3, decay=0.3, batch_size=batch_size, win=50, device=device)
    
    if delay_mode == 'nodelay':
        delay_name = ''
        snn = RSNN('shd', num_hidden=num_hidden, thresh=0.3, tau_m=tau_m, vreset=vreset, batch_size=batch_size, win=time_window, device=device)
    else:
        delay_name = '_delay'
        snn = RSNN_d('shd', num_hidden=num_hidden, thresh=0.3, tau_m=tau_m, vreset=vreset, batch_size=batch_size, win=time_window, device=device)

    snn.to(device)
    
    if tau_m != 'adp':
        tau_name = ''
    else:
        tau_name = '_adp'
        
    if vreset==0:
        vreset_name = ''
    else:
        vreset_name = '_vreset'
        
    # training configuration
    modelname = 'shd_rnn_{}{}{}{}.t7'.format(snn.num_hidden, tau_name, delay_name, vreset_name)
    
    print("-------TRAINING {} ---------".format(modelname))
    num_samples = train_dataset.images.size()[0]

    # super pythonic way to extract the parameters that will have 'normal' learning rate
    base_params = [getattr(snn,name.split('.')[0]).weight for name, _ in snn.state_dict().items() if name[0]=='f']

    # setting different learning rate for tau_m, if neeeded
    if tau_m=='adp':
        optimizer = torch.optim.Adam([
            {'params': base_params},
            {'params': snn.tau_m_h, 'lr': learning_rate * 10.0}],
            lr=learning_rate)
    else:    
        optimizer = torch.optim.Adam([
            {'params': base_params}],
            lr=learning_rate)

    act_fun = ActFun.apply


    # training loop
    taus_m = []
    for epoch in range(num_epochs):
        print('Epoch [%d/%d]'  % (epoch + 1, num_epochs))
        start_time = time.time()
        snn.train_step(train_loader, optimizer=optimizer, criterion=nn.MSELoss(), num_samples = num_samples)
        t = time.time() - start_time
        print('Time elasped:', time.time() - start_time)

        # update learning rate
        optimizer = snn.lr_scheduler(optimizer, lr_decay_epoch=1)

        # weight and decay recording
        # taus_m.append((snn.tau_m_h.data.detach().clone(), snn.tau_m_o.data.detach().clone()))

        if (epoch+1) % 5 ==0:
            snn.test(test_loader, criterion=nn.MSELoss())
            snn.save_model(modelname)   

    with open('training_log', 'a') as logs:
        logs.write("\nFinished training {} epochs for {}, batch_size {}, time_per_epoch {} s".format(num_epochs, modelname, batch_size, t))   



-------TRAINING shd_rnn_128.t7 ---------
Epoch [1/200]
Step [10/31], Loss: 0.50669
Step [20/31], Loss: 0.48628
Step [30/31], Loss: 0.48065
Time elasped: 2.338125705718994
Epoch [2/200]
Step [10/31], Loss: 0.47402
Step [20/31], Loss: 0.46763
Step [30/31], Loss: 0.46392
Time elasped: 2.3072896003723145
Epoch [3/200]
Step [10/31], Loss: 0.45579
Step [20/31], Loss: 0.45431
Step [30/31], Loss: 0.45088
Time elasped: 2.3101463317871094
Epoch [4/200]
Step [10/31], Loss: 0.44632
Step [20/31], Loss: 0.44278
Step [30/31], Loss: 0.44066
Time elasped: 2.31925368309021
Epoch [5/200]
Step [10/31], Loss: 0.43811
Step [20/31], Loss: 0.43377
Step [30/31], Loss: 0.43154
Time elasped: 2.3092103004455566
Test Accuracy of the model on the test samples: 30.029
Epoch [6/200]
Step [10/31], Loss: 0.42685
Step [20/31], Loss: 0.42596
Step [30/31], Loss: 0.42593
Time elasped: 2.3319456577301025
Epoch [7/200]
Step [10/31], Loss: 0.42065
Step [20/31], Loss: 0.41928
Step [30/31], Loss: 0.41654
Time elasped: 2.4275629

Step [30/31], Loss: 0.26343
Time elasped: 2.2927510738372803
Epoch [59/200]
Step [10/31], Loss: 0.26227
Step [20/31], Loss: 0.26133
Step [30/31], Loss: 0.26325
Time elasped: 2.293663740158081
Epoch [60/200]
Step [10/31], Loss: 0.26224
Step [20/31], Loss: 0.25889
Step [30/31], Loss: 0.25845
Time elasped: 2.2760913372039795
Test Accuracy of the model on the test samples: 60.498
Epoch [61/200]
Step [10/31], Loss: 0.26244
Step [20/31], Loss: 0.26083
Step [30/31], Loss: 0.25600
Time elasped: 2.2751753330230713
Epoch [62/200]
Step [10/31], Loss: 0.25565
Step [20/31], Loss: 0.25583
Step [30/31], Loss: 0.26083
Time elasped: 2.2771546840667725
Epoch [63/200]
Step [10/31], Loss: 0.25381
Step [20/31], Loss: 0.26278
Step [30/31], Loss: 0.25690
Time elasped: 2.278498888015747
Epoch [64/200]
Step [10/31], Loss: 0.25653
Step [20/31], Loss: 0.25307
Step [30/31], Loss: 0.25723
Time elasped: 2.281766891479492
Epoch [65/200]
Step [10/31], Loss: 0.25987
Step [20/31], Loss: 0.25778
Step [30/31], Loss: 0.25

Step [10/31], Loss: 0.23936
Step [20/31], Loss: 0.24363
Step [30/31], Loss: 0.23509
Time elasped: 2.2737879753112793
Epoch [117/200]
Step [10/31], Loss: 0.23918
Step [20/31], Loss: 0.23997
Step [30/31], Loss: 0.23953
Time elasped: 2.2682621479034424
Epoch [118/200]
Step [10/31], Loss: 0.23775
Step [20/31], Loss: 0.24139
Step [30/31], Loss: 0.23729
Time elasped: 2.266584873199463
Epoch [119/200]
Step [10/31], Loss: 0.24072
Step [20/31], Loss: 0.23601
Step [30/31], Loss: 0.23779
Time elasped: 2.2797107696533203
Epoch [120/200]
Step [10/31], Loss: 0.24153
Step [20/31], Loss: 0.23841
Step [30/31], Loss: 0.24070
Time elasped: 2.2690815925598145
Test Accuracy of the model on the test samples: 62.158
Epoch [121/200]
Step [10/31], Loss: 0.24194
Step [20/31], Loss: 0.23670
Step [30/31], Loss: 0.24075
Time elasped: 2.268890619277954
Epoch [122/200]
Step [10/31], Loss: 0.24079
Step [20/31], Loss: 0.23929
Step [30/31], Loss: 0.23980
Time elasped: 2.2748360633850098
Epoch [123/200]
Step [10/31], Lo

Step [30/31], Loss: 0.23341
Time elasped: 2.2762057781219482
Epoch [174/200]
Step [10/31], Loss: 0.23796
Step [20/31], Loss: 0.23897
Step [30/31], Loss: 0.22894
Time elasped: 2.276337146759033
Epoch [175/200]
Step [10/31], Loss: 0.23493
Step [20/31], Loss: 0.24025
Step [30/31], Loss: 0.23074
Time elasped: 2.294680118560791
Test Accuracy of the model on the test samples: 62.256
Epoch [176/200]
Step [10/31], Loss: 0.23554
Step [20/31], Loss: 0.23457
Step [30/31], Loss: 0.23541
Time elasped: 2.273869037628174
Epoch [177/200]
Step [10/31], Loss: 0.23407
Step [20/31], Loss: 0.23684
Step [30/31], Loss: 0.23560
Time elasped: 2.276886463165283
Epoch [178/200]
Step [10/31], Loss: 0.23662
Step [20/31], Loss: 0.23482
Step [30/31], Loss: 0.23095
Time elasped: 2.2846732139587402
Epoch [179/200]
Step [10/31], Loss: 0.23789
Step [20/31], Loss: 0.23392
Step [30/31], Loss: 0.23158
Time elasped: 2.327153205871582
Epoch [180/200]
Step [10/31], Loss: 0.23259
Step [20/31], Loss: 0.23566
Step [30/31], Loss:

Test Accuracy of the model on the test samples: 53.516
Epoch [31/200]
Step [10/31], Loss: 0.31091
Step [20/31], Loss: 0.31147
Step [30/31], Loss: 0.31164
Time elasped: 2.2760534286499023
Epoch [32/200]
Step [10/31], Loss: 0.31165
Step [20/31], Loss: 0.30988
Step [30/31], Loss: 0.31158
Time elasped: 2.2684481143951416
Epoch [33/200]
Step [10/31], Loss: 0.31152
Step [20/31], Loss: 0.30989
Step [30/31], Loss: 0.30467
Time elasped: 2.2769832611083984
Epoch [34/200]
Step [10/31], Loss: 0.30447
Step [20/31], Loss: 0.30979
Step [30/31], Loss: 0.30592
Time elasped: 2.3042755126953125
Epoch [35/200]
Step [10/31], Loss: 0.30596
Step [20/31], Loss: 0.30389
Step [30/31], Loss: 0.30511
Time elasped: 2.32059907913208
Test Accuracy of the model on the test samples: 55.322
Epoch [36/200]
Step [10/31], Loss: 0.31456
Step [20/31], Loss: 0.30767
Step [30/31], Loss: 0.30303
Time elasped: 2.3044447898864746
Epoch [37/200]
Step [10/31], Loss: 0.30454
Step [20/31], Loss: 0.30142
Step [30/31], Loss: 0.30090
T

...

Step [10/31], Loss: 0.02243
Step [20/31], Loss: 0.02170
Step [30/31], Loss: 0.02269
Time elasped: 3.7426793575286865
Epoch [153/200]
Step [10/31], Loss: 0.02290
Step [20/31], Loss: 0.02145
Step [30/31], Loss: 0.02215
Time elasped: 3.7463338375091553
Epoch [154/200]
Step [10/31], Loss: 0.02204
Step [20/31], Loss: 0.02238
Step [30/31], Loss: 0.02175
Time elasped: 3.7251224517822266
Epoch [155/200]
Step [10/31], Loss: 0.02205
Step [20/31], Loss: 0.02220
Step [30/31], Loss: 0.02204
Time elasped: 3.737946033477783
Test Accuracy of the model on the test samples: 80.566
Epoch [156/200]
Step [10/31], Loss: 0.02190
Step [20/31], Loss: 0.02304
Step [30/31], Loss: 0.02136
Time elasped: 3.715266704559326
Epoch [157/200]
Step [10/31], Loss: 0.02158
Step [20/31], Loss: 0.02173
Step [30/31], Loss: 0.02216
Time elasped: 3.728848457336426
Epoch [158/200]
Step [10/31], Loss: 0.02134
Step [20/31], Loss: 0.02272
Step [30/31], Loss: 0.02129
Time elasped: 3.748340606689453
Epoch [159/200]
Step [10/31], Loss

Step [10/31], Loss: 0.23913
Step [20/31], Loss: 0.23984
Step [30/31], Loss: 0.23622
Time elasped: 3.7485146522521973
Epoch [10/200]
Step [10/31], Loss: 0.23522
Step [20/31], Loss: 0.22041
Step [30/31], Loss: 0.20788
Time elasped: 3.739809036254883
Test Accuracy of the model on the test samples: 71.143
Epoch [11/200]
Step [10/31], Loss: 0.20186
Step [20/31], Loss: 0.20212
Step [30/31], Loss: 0.20615
Time elasped: 3.713948965072632
Epoch [12/200]
Step [10/31], Loss: 0.20241
Step [20/31], Loss: 0.19568
Step [30/31], Loss: 0.19627
Time elasped: 3.746553421020508
Epoch [13/200]
Step [10/31], Loss: 0.19041
Step [20/31], Loss: 0.21344
Step [30/31], Loss: 0.20599
Time elasped: 3.7346718311309814
Epoch [14/200]
Step [10/31], Loss: 0.20046
Step [20/31], Loss: 0.18662
Step [30/31], Loss: 0.17823
Time elasped: 3.73227858543396
Epoch [15/200]
Step [10/31], Loss: 0.17059
Step [20/31], Loss: 0.17388
Step [30/31], Loss: 0.17773
Time elasped: 3.753124475479126
Test Accuracy of the model on the test sam

Step [30/31], Loss: 0.03709
Time elasped: 3.7088513374328613
Epoch [67/200]
Step [10/31], Loss: 0.03369
Step [20/31], Loss: 0.03482
Step [30/31], Loss: 0.03391
Time elasped: 3.731590509414673
Epoch [68/200]
Step [10/31], Loss: 0.03404
Step [20/31], Loss: 0.03352
Step [30/31], Loss: 0.03300
Time elasped: 3.7469606399536133
Epoch [69/200]
Step [10/31], Loss: 0.03290
Step [20/31], Loss: 0.03328
Step [30/31], Loss: 0.03434
Time elasped: 3.732672691345215
Epoch [70/200]
Step [10/31], Loss: 0.03341
Step [20/31], Loss: 0.03448
Step [30/31], Loss: 0.03421
Time elasped: 3.7274041175842285
Test Accuracy of the model on the test samples: 82.422
Epoch [71/200]
Step [10/31], Loss: 0.03440
Step [20/31], Loss: 0.03397
Step [30/31], Loss: 0.03385
Time elasped: 3.7094621658325195
Epoch [72/200]
Step [10/31], Loss: 0.03301
Step [20/31], Loss: 0.03210
Step [30/31], Loss: 0.03361
Time elasped: 3.7424283027648926
Epoch [73/200]
Step [10/31], Loss: 0.03273
Step [20/31], Loss: 0.03315
Step [30/31], Loss: 0.0

Step [10/31], Loss: 0.02570
Step [20/31], Loss: 0.02649
Step [30/31], Loss: 0.02585
Time elasped: 3.7269999980926514
Epoch [125/200]
Step [10/31], Loss: 0.02603
Step [20/31], Loss: 0.02616
Step [30/31], Loss: 0.02494
Time elasped: 3.7481842041015625
Test Accuracy of the model on the test samples: 81.494
Epoch [126/200]
Step [10/31], Loss: 0.02564
Step [20/31], Loss: 0.02534
Step [30/31], Loss: 0.02570
Time elasped: 3.6997625827789307
Epoch [127/200]
Step [10/31], Loss: 0.02523
Step [20/31], Loss: 0.03445
Step [30/31], Loss: 0.03531
Time elasped: 3.7478690147399902
Epoch [128/200]
Step [10/31], Loss: 0.03081
Step [20/31], Loss: 0.02883
Step [30/31], Loss: 0.02636
Time elasped: 3.7269833087921143
Epoch [129/200]
Step [10/31], Loss: 0.02649
Step [20/31], Loss: 0.02641
Step [30/31], Loss: 0.02732
Time elasped: 3.748117446899414
Epoch [130/200]
Step [10/31], Loss: 0.02688
Step [20/31], Loss: 0.02590
Step [30/31], Loss: 0.02585
Time elasped: 3.728456974029541
Test Accuracy of the model on th

Step [10/31], Loss: 0.02368
Step [20/31], Loss: 0.02369
Step [30/31], Loss: 0.02321
Time elasped: 3.717550039291382
Epoch [182/200]
Step [10/31], Loss: 0.02327
Step [20/31], Loss: 0.02299
Step [30/31], Loss: 0.02413
Time elasped: 3.733454704284668
Epoch [183/200]
Step [10/31], Loss: 0.02373
Step [20/31], Loss: 0.02320
Step [30/31], Loss: 0.02368
Time elasped: 3.735863447189331
Epoch [184/200]
Step [10/31], Loss: 0.02351
Step [20/31], Loss: 0.02340
Step [30/31], Loss: 0.02290
Time elasped: 3.732646942138672
Epoch [185/200]
Step [10/31], Loss: 0.02306
Step [20/31], Loss: 0.02364
Step [30/31], Loss: 0.02313
Time elasped: 3.741403818130493
Test Accuracy of the model on the test samples: 81.787
Epoch [186/200]
Step [10/31], Loss: 0.02213
Step [20/31], Loss: 0.02364
Step [30/31], Loss: 0.02372
Time elasped: 3.6975998878479004
Epoch [187/200]
Step [10/31], Loss: 0.02330
Step [20/31], Loss: 0.02352
Step [30/31], Loss: 0.02340
Time elasped: 3.7325644493103027
Epoch [188/200]
Step [10/31], Loss:

In [20]:
vreset

0

In [12]:
m.load_model(modelname, 256, device)